In [1]:
import os

import numpy as np
from archetypal import settings
from pathlib import Path
from uuid import uuid4

from shoeboxer.builder import ShoeboxConfiguration, ShoeBox
from shoeboxer.schedules import default_schedules

# Check if we are running on Windows or Linux using os
if os.name == "nt":
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("C:/EnergyPlusV22-2-0")
else:
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("/usr/local/EnergyPlus-22-2-0")

c:\tools\Anaconda3\envs\ml-for-bem-final\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# epw_path = "C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw"
epw_path ="C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_NY_New.York-LaGuardia.AP.725030_TMYx.2004-2018.epw"
config = {
    "FacadeMass": 1,
    "RoofMass": 1,
    "FacadeRValue": 4.88,
    "RoofRValue": 4.88,
    "SlabRValue": 4.88,
    "WindowUValue": 2.0,
    "WindowShgc": 0.75,
    "PeopleDensity": 0.025,
    "EquipmentPowerDensity": 2,
    "LightingPowerDensity": 2,
    "HeatingSetpoint": 19,
    "CoolingSetpoint": 25,
    "cop_heating": 1,
    "cop_cooling": 1,
    "VentilationMode": 0,
    "VentilationPerArea": 0.001,
    "VentilationPerPerson": 0.001,
    "EconomizerSettings": 0,
    "RecoverySettings": 0,
    "Infiltration": 0.0005,
    "wwr": 0.2,
    "roof_2_footprint": 1,
    "ground_2_footprint": 0,
    "orientation": 0,
    "width": 3,
    "height": 4,
    "perim_depth": 3.6,
    "core_depth": 6.4,
    "building_id": 0,
    "weight": 1,
    "template_idx": 0,
}

"""
Make Schedules
"""

# scheds = default_schedules[0]
sin_schedule = np.linspace(0,2*np.pi*365-1/24*2*np.pi,365*24)
sin_schedule = -np.cos(sin_schedule)/2 + 0.5
scheds = np.stack([sin_schedule]*3).reshape(3,8760)

"""
Make Shoebox
"""
sb_config = ShoeboxConfiguration()
sb_config.width = config["width"]
sb_config.height = config["height"]
sb_config.adiabatic_partition_flag = 0
sb_config.perim_depth = config["perim_depth"]
sb_config.core_depth = config["core_depth"]
sb_config.wwr = config["wwr"]
sb_config.roof_2_footprint = config["roof_2_footprint"]
sb_config.ground_2_footprint = config["ground_2_footprint"]
sb_config.shading_vect = np.zeros(12)
sb_config.orientation = config["orientation"]
template_dict = config.copy()
template_dict["VentilationMode"] = 0
template_dict["schedules"] = scheds

sb = ShoeBox(
    name=str(uuid4()),
    epw=epw_path,
    shoebox_config=sb_config,
    template_dict=template_dict,
    output_directory="data/sim_results/"
)
idf = sb.idf(run_simulation=False)

"""
Simulate
"""

hourly_df, monthly_df = sb.simulate(idf=idf)
df = sb.postprocess(monthly_df)
annual = df.T.groupby(level=[0,1]).sum().T
annual

INFO:shoeboxer.geometry_utils:New perimeter depth: 3.6
INFO:shoeboxer.geometry_utils:New core depth: 6.4
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving -1.4 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.geometry_utils:Maximum shading point is 0.0
INFO:shoeboxer.geometry_utils:Placing shading around center point [1.5 0. ]
INFO:ShoeBox:Building idf for data\sim_results\f82b510d-2354-47dc-a0e8-333916bdbe4e.epjson
INFO:ShoeBox:EnergyPlus Starting
INFO:ShoeBox:EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.11.17 16:20
INFO:ShoeBox:Converted input file format. Exiting.
INFO:ShoeBox:EnergyPlus Run Time=00hr 00min  0.13sec
INFO:ShoeBox:Command executed successfully.
C:\EnergyPlusV22-2-0\ExpandObjects.EXE #0-f82b510d-2354-47dc-a0e8-333916bdbe4e.idf: 3.00it [00:00, 9.05it/s]
C:\EnergyPlusV22-2-0\energyplus.exe f82b510d-2354-47dc-a0e8-333916bdbe4e.idf: 55it [00:04, 13.38it/s]


KeyValue   Name   
Core       Cooling    16.728359
           Heating    23.218456
Perimeter  Cooling    51.524641
           Heating    39.305976
dtype: float64